In [2]:

%run ../py/sql_utlis.py
sqlu = SqlUtilities()
conn, cursor = sqlu.get_jh_conn_cursor()
isheaders_df = pd.DataFrame(sqlu.get_execution_results(cursor, sqlu.select_filename_isheader_sql_str,
                                                     verbose=False))
isheaders_df.head(5)

,file_name,is_header
0,(Lead_Data_Scientist)_Lead_Product_Analyst_-_C...,None
1,(Lead_Data_Scientist)_Lead_Product_Analyst_-_C...,False
2,(Lead_Data_Scientist)_Lead_Product_Analyst_-_C...,None
3,(Lead_Data_Scientist)_Lead_Product_Analyst_-_C...,False
4,(Lead_Data_Scientist)_Lead_Product_Analyst_-_C...,True


In [3]:

def f(df):
    mask_series = df.is_header.isnull()

    return df[mask_series].shape[0]
isheaders_series = isheaders_df.groupby('file_name').apply(f).sort_values()
isheaders_series

file_name
Data_Scientist_I_-_Generalist_4db73ec6324709a1.html                                         0
Sr._Data_Scientist_-_Chicago,_IL_60661_-_Indeed.com_c6f867b4166b93af.html                   0
Data_Scientist_-_New_York,_NY_10016_-_Indeed.com_945e6b454157cd18.html                      0
Research_Data_Analyst_Statistician_-_Washington,_DC_-_Indeed.com_aed1c2970bb86669.html      0
Data_Scientist_-_San_Francisco_Bay_Area,_CA_-_Indeed.com_3a7ad5d4b8526b01.html              0
                                                                                         ... 
Sr._Data_Scientist_efa5ad2d82bd1fcd.html                                                  177
Data_Scientist_00ba1a22ba67ffd2.html                                                      182
Data_Scientist_2c882a7cc4be1655.html                                                      188
NLP_Data_Scientist_544ed6cdaab8d764.html                                                  192
Senior_Data_Scientist_c57ded0649391c68.html       

In [ ]:

sqlu.build_child_strs_list_dictionary(cursor, verbose=False)
[sent_str for sublist in sqlu.CHILD_STRS_LIST_DICT.values() for sent_str in sublist]

In [2]:

def f(x):
    
    return 'bachelor' in x.lower()
mask_series = bachelor_df.navigable_parent.map(f)
bachelor_df[mask_series].to_dict()

{'navigable_parent': {73: '<li>Bachelor’s degree in Computer Information Science or related technical field required, Graduate degree preferred. May consider equivalent work experience.</li>',
  74: '<li>Bachelor’s degree or equivalent experience in computer science, mathematics, statistics, economics, or similar</li>',
  141: '<p class="icl-u-xs-my--none">Bachelor\'s (Preferred)</p>'}}

In [4]:

sql_str = r"""UPDATE NavigableParents
    SET
        is_minimum_qualification = 0,
        is_preferred_qualification = 0,
        is_educational_requirement = 1
    WHERE (navigable_parent LIKE ? ESCAPE '\')"""
for tag_str in bachelor_df[mask_series].navigable_parent:
    cursor_obj = cursor.execute(sql_str, (sqlu.wc_rgx.sub(r'\\\g<1>', tag_str),))
    cursor.commit()